In [48]:
%load_ext autoreload

In [94]:
%autoreload
import Eveo
import functools, math

In [95]:
Eveo.Loadin()

Loading Blueprint Data - 1/4
Loading Item Data - 2/4
Loading Planet Data - 3/4
Fetching Prices - 4/4
Done Loadin


In [3]:
Eveo.piData

{65: {'cycleTime': 3600,
  'nameID': {'de': 'Superconductors',
   'en': 'Superconductors',
   'fr': 'Superconducteurs',
   'ja': '超電導体',
   'ko': '초전도체',
   'ru': 'Superconductors',
   'zh': '超导体'},
  'pins': [2470, 2472, 2474, 2480, 2484, 2485, 2491, 2494],
  'types': {2389: {'isInput': True, 'quantity': 40},
   3645: {'isInput': True, 'quantity': 40},
   9838: {'isInput': False, 'quantity': 5}}},
 66: {'cycleTime': 3600,
  'nameID': {'de': 'Coolant',
   'en': 'Coolant',
   'fr': 'Réfrigérant',
   'ja': '冷却材',
   'ko': '냉각수',
   'ru': 'Coolant',
   'zh': '冷却剂'},
  'pins': [2470, 2472, 2474, 2480, 2484, 2485, 2491, 2494],
  'types': {2390: {'isInput': True, 'quantity': 40},
   3645: {'isInput': True, 'quantity': 40},
   9832: {'isInput': False, 'quantity': 5}}},
 67: {'cycleTime': 3600,
  'nameID': {'de': 'Rocket Fuel',
   'en': 'Rocket Fuel',
   'fr': 'Carburant de roquette',
   'ja': 'ロケット燃料',
   'ko': '로켓 연료',
   'ru': 'Rocket Fuel',
   'zh': '火箭燃料'},
  'pins': [2470, 2472, 2474, 24

In [101]:
PI={}
piMarketGroups = {1335: "Refined Commodities - Tier 2", 1334: "Refined Commodities - Tier 2", 1336: 'Specialized Commodities - Tier 3', 1334: "Basic Commodities - Tier 1"}
piMarketGroupsT = {1335: 2, 1334: 1, 1336: 3, 1337: 4, 1042:1}
piTechExportMultiplier = {4:1,3:1,2:7200.00,1:400}
piTechImportMultiplier = {4:1,3:1,2:7200.00,1:400}

for pi_schematicID in Eveo.piData:
    pi_schematic = Eveo.piData[pi_schematicID]
    PIMat = {'inputs':[],'typeID':0,'amount':0,'name':pi_schematic['nameID']['en']}
    PIMat['cycleTimeMinutes'] = pi_schematic['cycleTime']
    PIMat['cycleTimeHours'] = PIMat['cycleTimeMinutes']/ 3600
    for typeID in pi_schematic['types']:
        typedata = pi_schematic['types'][typeID]
        if(typedata['isInput'] == True):
            PIMat['inputs'].append({'typeID':typeID,'amount':typedata['quantity'],'name':Eveo.NameOf(typeID)})
        else:
            PIMat['typeID'] = typeID
            PIMat['amount'] = typedata['quantity']
    for inputM in PIMat['inputs']:
        inputM['perhour'] =  inputM['amount'] / PIMat['cycleTimeHours']
    typeID = PIMat['typeID']
    PIMat['amountperHour'] =  PIMat['amount'] / PIMat['cycleTimeHours']
    PIMat['priceEach'] = Eveo.GetPrice(str(typeID))
    PIMat['priceForRunQty'] = PIMat['priceEach'] * PIMat['amount']
    PIMat['materialCostforRun'] = math.ceil(functools.reduce(lambda a, b: a+(Eveo.GetPrice(str(b['typeID'])) *  b['amount'] ), PIMat['inputs'],0))
    PIMat['materialCostEach'] = PIMat['materialCostforRun']  /  PIMat['amount']
    PIMat['profitRun'] = PIMat['priceForRunQty'] / PIMat['materialCostforRun']
    PIMat['profitEach'] =  PIMat['priceEach'] /  PIMat['materialCostEach']
    PIMat['profitIskRun'] = math.floor(PIMat['priceForRunQty'] - PIMat['materialCostforRun'])
    PIMat['profitIskEach'] = math.floor( PIMat['priceEach'] -  PIMat['materialCostEach'])
    PIMat['tech'] = piMarketGroupsT[Eveo.typeData[int(typeID)]['marketGroupID']]
    PIMat['shortName'] = PIMat['name'].replace(" ", "").replace("er", "r").replace("al", "l").replace("an", "n").replace("ro", "r").replace("ature","").replace("es","s").replace("en","n")
    PI[PIMat['typeID']] = PIMat
PI

{9838: {'inputs': [{'typeID': 2389,
    'amount': 40,
    'name': 'Plasmoids',
    'perhour': 40.0},
   {'typeID': 3645, 'amount': 40, 'name': 'Water', 'perhour': 40.0}],
  'typeID': 9838,
  'amount': 5,
  'name': 'Superconductors',
  'cycleTimeMinutes': 3600,
  'cycleTimeHours': 1.0,
  'amountperHour': 5.0,
  'priceEach': 8027.08,
  'priceForRunQty': 40135.4,
  'materialCostforRun': 36922,
  'materialCostEach': 7384.4,
  'profitRun': 1.0870321217702184,
  'profitEach': 1.0870321217702184,
  'profitIskRun': 3213,
  'profitIskEach': 642,
  'tech': 2,
  'shortName': 'Suprconductors'},
 9832: {'inputs': [{'typeID': 2390,
    'amount': 40,
    'name': 'Electrolytes',
    'perhour': 40.0},
   {'typeID': 3645, 'amount': 40, 'name': 'Water', 'perhour': 40.0}],
  'typeID': 9832,
  'amount': 5,
  'name': 'Coolant',
  'cycleTimeMinutes': 3600,
  'cycleTimeHours': 1.0,
  'amountperHour': 5.0,
  'priceEach': 10083.84,
  'priceForRunQty': 50419.2,
  'materialCostforRun': 36459,
  'materialCostEach'

In [102]:
with open('PI_DATA.yaml', "w") as stream:
    try:
        Eveo.yaml.safe_dump(PI,stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
#Have a bank of 12(3*4) planets. Each can extract X/Hr, of TWO of the planet's T0 resource types. 
#T0 always needs refined into a T1 ouput material, so the T0 to T1 process is ignored and just shown as T1. 
#Thefore the output rate of the planet is measured by how many active refineries are running per hour per resource.
#1planet, max 3 refineries per resource. 12 planets = 12 * (3*2) = 72 total refineries. This is the 'Stock' supply

In [ ]:
#T1 through T5 can be bought on the market. Combining these together generates TN+1. These sell for money.
#The Stock supply of T1 can also be sold. However as these were gained 'at no cost', 
#the biggest profits are to be made by utilising stock T1's and reducing Market Aquisitions.


In [ ]:
#For every TN->TN+1 process, a 'factory slot' must be used.
#There is a limit of 5 factory planets. Each can house x6 Factories. 30 Factory slots total.
#Given 30 factory slots, and 12 harvest-stock planets. What is the most lucrative setup?

In [103]:
lasttech = 4
for p in list(sorted(list(PI.values()),key=lambda thing:(-thing["tech"],thing["name"]), reverse=False )):
    if(not p["tech"] == lasttech): print(" ")
    lasttech = p["tech"] 
    print(p["tech"],str(p["typeID"]).ljust(6),Eveo.formatPrice(p["priceEach"],0).ljust(10),p["name"][:21].ljust(21),"\t",p["shortName"])

4 2867   1,613,557  Broadcast Node        	 BradcastNode
4 2868   1,644,427  Integrity Response Dr 	 IntegrityRsponseDrns
4 2869   727,124    Nano-Factory          	 Nno-Factory
4 2870   757,502    Organic Mortar Applic 	 OrgnicMortarApplicators
4 2871   1,035,262  Recursive Computing M 	 RecursiveComputingModule
4 2872   1,492,749  Self-Harmonizing Powe 	 Self-HarmonizingPowrCore
4 2875   979,328    Sterile Conduits      	 StrileConduits
4 2876   1,663,998  Wetware Mainframe     	 WetwareMainframe
 
3 2358   67,127     Biotech Research Repo 	 BiotechRsearchReports
3 2345   70,362     Camera Drones         	 CamraDrns
3 2344   36,708     Condensates           	 Condnsats
3 2367   101,596    Cryoprotectant Soluti 	 CryoprtectntSolution
3 17392  78,735     Data Chips            	 DataChips
3 2348   98,514     Gel-Matrix Biopaste   	 Gel-MatrixBiopaste
3 9834   61,245     Guidance Systems      	 GuidnceSystems
3 2366   58,500     Hazmat Detection Syst 	 HazmatDetectionSystems
3 2361   79,

In [92]:
list(PI.values())

[]

In [71]:
list(sorted(list(PI.values()),key=lambda thing:thing["tech"], reverse=True ))

[{'inputs': [{'typeID': 2344,
    'amount': 6,
    'name': 'Condensates',
    'perhour': 6.0},
   {'typeID': 2393, 'amount': 40, 'name': 'Bacteria', 'perhour': 40.0},
   {'typeID': 9848, 'amount': 6, 'name': 'Robotics', 'perhour': 6.0}],
  'typeID': 2870,
  'amount': 1,
  'name': 'Organic Mortar Applicators',
  'cycleTimeMinutes': 3600,
  'cycleTimeHours': 1.0,
  'amountperHour': 1.0,
  'priceEach': 757501.56,
  'priceForRunQty': 757501.56,
  'materialCostforRun': 612556,
  'materialCostEach': 612556.0,
  'profitRun': 1.236624178034335,
  'profitEach': 1.236624178034335,
  'profitIskRun': 144945,
  'profitIskEach': 144945,
  'tech': 4,
  'shortName': 'OrgnicMortarApplicators'},
 {'inputs': [{'typeID': 2351,
    'amount': 6,
    'name': 'Smartfab Units',
    'perhour': 6.0},
   {'typeID': 3645, 'amount': 40, 'name': 'Water', 'perhour': 40.0},
   {'typeID': 28974, 'amount': 6, 'name': 'Vaccines', 'perhour': 6.0}],
  'typeID': 2875,
  'amount': 1,
  'name': 'Sterile Conduits',
  'cycleTim

In [100]:
Eveo.typeData[2393]

{'basePrice': 1.0,
 'description': {'de': 'Der Begriff "Bakterien" umfasst eine große, facettenreiche Familie einzelliger Mikroorganismen, von denen manche in beinahe jedem Klima New Edens gefunden werden können und andere sich nur in den Körpern anderer Lebensformen wohlfühlen. Obwohl einige Bakterien dafür bekannt sind, Krankheiten zu übertragen, gibt es auch andere, die den Menschen mehr nutzen als schaden.',
  'en': 'The term “bacteria” covers a wide, diverse family of unicellular microorganisms, from those found in almost every climate in New Eden to those that thrive in the bodies of other living beings. Though some bacteria are known to convey diseases, others are more helpful than harmful to humans.',
  'fr': "Le terme «\xa0bactérie\xa0» englobe une large famille de micro-organismes unicellulaires, de ceux que l'on trouve dans presque tous les climats de New Eden à ceux qui se développent dans le corps d'autres êtres vivants. Si certaines bactéries sont réputées pour transmettr